<a href="https://colab.research.google.com/github/chakri9m/Chakri/blob/main/lerobot/training-deffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 x 🦾: Training pusht with LeRobot Notebook

Welcome to the **LeRobot pusht training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `pusht` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `diffusion-pusht` policy for 20,000 steps typically takes **about 5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer!

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [3]:
!git clone https://github.com/huggingface.co/lerobot/pusht.git
!cd pusht
!cd lerobot && pip install -e .


Cloning into 'pusht'...
remote: Not Found
fatal: repository 'https://github.com/huggingface.co/lerobot/pusht.git/' not found
/bin/bash: line 1: cd: pusht: No such file or directory
/bin/bash: line 1: cd: lerobot: No such file or directory


In [4]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 29669, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 29669 (delta 0), reused 0 (delta 0), pack-reused 29664 (from 1)
Receiving objects: 100% (29669/29669), 161.17 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (19222/19222), done.
Filtering content: 100% (45/45), 69.04 MiB | 18.07 MiB/s, done.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ffmpeg=7.1.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attr-2.5.1                 |       h166bdaf_1          69 KB  conda-forge
    ca-certif

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [5]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chakravardhan999 (chakravardhan999-sheffield-hallam-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Install pusht dependencies

In [6]:
!pip install -e lerobot


Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.1.0-0.editable-py3-none-any.whl size=16698 sha256=86976ff25df68366ef418acfa6a1159137657fcb31b8c0104fa1263ba8b50d94
  Stored in directory: /tmp/pip-ephem-wheel-cache-s__qogk7/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8338
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.1.0
    Uninstalling lerobot-0.1.0:
      Successfully uninstalled lerobot-0.1.0


In [7]:
!cd lerobot && pip install -e


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-e option requires 1 argument


In [8]:
!ls /content/lerobot


benchmarks	    docker    LICENSE	   media	   src
CODE_OF_CONDUCT.md  docs      Makefile	   pyproject.toml  tests
CONTRIBUTING.md     examples  MANIFEST.in  README.md


In [9]:
!ls /content/lerobot/examples


1_load_lerobot_dataset.py	 4_train_policy_with_script.md
2_evaluate_pretrained_policy.py  backward_compatibility
3_train_policy.py		 lekiwi


In [10]:
!find /content/lerobot -name train.py


/content/lerobot/src/lerobot/scripts/train.py
/content/lerobot/src/lerobot/configs/train.py


In [11]:
!pip install datasets



In [23]:
!python /content/lerobot/src/lerobot/scripts/train.py\
    --output_dir=outputs/train/diffusion_pusht \
    --policy.type=diffusion \
    --dataset.repo_id=lerobot/pusht \
    --seed=100000 \
    --env.type=pusht \
    --batch_size=64 \
    --steps=200000 \
    --eval_freq=25000 \
    --save_freq=25000 \
    --wandb.enable=true \
    --policy.repo_id=local



INFO 2025-07-18 11:48:44 ils/utils.py:48 Cuda backend detected, using cuda.
WARNING 2025-07-18 11:48:44 /policies.py:79 Device 'None' is not available. Switching to 'cuda'.
Traceback (most recent call last):
  File "/content/lerobot/src/lerobot/scripts/train.py", line 291, in <module>
    train()
  File "/content/lerobot/src/lerobot/configs/parser.py", line 225, in wrapper_inner
    response = fn(cfg, *args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/lerobot/src/lerobot/scripts/train.py", line 110, in train
    cfg.validate()
  File "/content/lerobot/src/lerobot/configs/train.py", line 101, in validate
    raise FileExistsError(
FileExistsError: Output directory outputs/train/diffusion_pusht already exists and resume is False. Please change your output directory so that outputs/train/diffusion_pusht is not overwritten.


In [14]:
!pip install gym-pusht


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 145.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 135.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 63.1 MB/s eta 0:00:00


In [15]:
!pip install -e ".[aloha, pusht]"

Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [17]:
!python /content/lerobot/src/lerobot/scripts/eval.py --your_arguments_here


usage: eval.py [-h] [--config_path str] [--env str] [--env.obs_type str]
               [--env.render_mode str] [--env.visualization_width str]
               [--env.visualization_height str] [--robot str]
               [--env.robot.type str] [--teleop str] [--env.teleop.type str]
               [--env.task str] [--env.fps str] [--env.features str]
               [--env.features_map str] [--env.type str] [--env.name str]
               [--env.use_viewer str] [--env.gripper_penalty str]
               [--env.use_gamepad str] [--env.state_dim str]
               [--env.action_dim str] [--env.episode_length str]
               [--video_record str] [--env.video_record.enabled str]
               [--env.video_record.record_dir str]
               [--env.video_record.trajectory_name str]
               [--env.reward_classifier_pretrained_path str]
               [--robot_config str] [--env.robot_config.type str]
               [--teleop_config str] [--env.teleop_config.type str]
           

In [21]:
!python /content/lerobot/examples/2_evaluate_pretrained_policy.py \
    --policy.path=outputs/train/diffusion_pusht/checkpoints/175000/pretrained_model \
    --env.type=pusht \
    --eval.n_episodes=500 \
    --eval.batch_size=50


{'observation.image': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 96, 96)), 'observation.state': PolicyFeature(type=<FeatureType.STATE: 'STATE'>, shape=(2,))}
Dict('agent_pos': Box(0.0, 512.0, (2,), float64), 'pixels': Box(0, 255, (96, 96, 3), uint8))
{'action': PolicyFeature(type=<FeatureType.ACTION: 'ACTION'>, shape=(2,))}
Box(0.0, 512.0, (2,), float32)
Traceback (most recent call last):
  File "/content/lerobot/examples/2_evaluate_pretrained_policy.py", line 70, in <module>
    numpy_observation, info = env.reset(seed=42)
                              ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gymnasium/wrappers/time_limit.py", line 75, in reset
    return self.env.reset(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gymnasium/wrappers/order_enforcing.py", line 61, in reset
    return self.env.reset(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gymn

In [20]:

from IPython.display import Video

Video("/content/lerobot/outputs/eval/example_pusht_diffusion/rollout.mp4", embed=True, width=600)


In [22]:
!python /content/lerobot/src/lerobot/scripts/eval.py \
    --policy.path=outputs/train/diffusion_pusht/checkpoints/175000/pretrained_model \
    --output_dir=outputs/eval/diffusion_pusht/175000 \
    --env.type=pusht \
    --eval.n_episodes=500 \
    --eval.batch_size=50


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/draccus/parsers/decoding.py", line 159, in decode_dataclass
    instance = origin(**init_args)
               ^^^^^^^^^^^^^^^^^^^
  File "<string>", line 9, in __init__
  File "/content/lerobot/src/lerobot/configs/eval.py", line 43, in __post_init__
    self.policy = PreTrainedConfig.from_pretrained(policy_path, cli_overrides=cli_overrides)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/lerobot/src/lerobot/configs/policies.py", line 172, in from_pretrained
    config_file = hf_hub_download(
                  ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/utils/_validators.py", line 106, in _inner_fn
    validate_repo_id(arg_value)
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/utils/_validators.py", line 154, in validate_repo_id
    raise HFValidationError(
huggingface_hub.errors.HFValidation

In [ ]:
!python /content/lerobot/src/lerobot/scripts/eval.py \
    --policy.path=outputs/train/diffusion_pusht/checkpoints/175000/pretrained_model \
    --output_dir=outputs/eval/diffusion_pusht/175000 \
    --env.type=pusht \
    --eval.n_episodes=500 \
    --eval.batch_size=50 \
    --device=cuda \
    --use_amp=false

In [ ]:
%cd /content/lerobot
!pip install -e .

In [ ]:
!ls -R /content/lerobot | head -n 100



part 2

In [ ]:
!ls /content/lerobot/lerobot/scripts

In [ ]:
!python /content/lerobot/lerobot/scripts/eval.py  --help


In [ ]:
from huggingface_hub import notebook_login
notebook_login()



In [ ]:
import json
import os

info = {
    "codebase_version": "0.0.1",
    "features": {
        "image": {
            "dtype": "uint8",
            "shape": [128, 128, 3]
        },
        "action": {
            "dtype": "float32",
            "shape": [7]
        }
    }
}

os.makedirs("/content/move_to_box/meta", exist_ok=True)

with open("/content/move_to_box/meta/info.json", "w") as f:
    json.dump(info, f, indent=2)

print("✅ Fixed info.json written.")


In [ ]:
!git clone https://huggingface.co/Frontier-Machines/move_to_box


In [ ]:
!cd move_to_box && ls -l



In [ ]:
!head -40 move_to_box/README.md


In [ ]:
from lerobot.policies.diffusion.modeling_diffusion import DiffusionPolicy



In [ ]:
# Change this to any of the supported ones from HuggingFace
!pretrained_policy_path = "lerobot/diffusion_open_drawer"

!policy = DiffusionPolicy.from_pretrained(pretrained_policy_path)


In [ ]:
!pip install lerobot


In [ ]:
ls -l /content/movetobox.py


In [ ]:
import lerobot
import pkgutil

print("lerobot submodules:")
for mod in pkgutil.iter_modules(lerobot.__path__):
    print(mod.name)


In [ ]:
!python /content/drive/MyDrive/movetobox.py


In [ ]:
!ls /content/lerobot/src



In [ ]:
import sys
sys.path.append('/content/lerobot/src')


WHy?

In [ ]:
!python -m lerobot.datasets.v2.convert_dataset_v1_to_v2 \
    --repo-id Frontier-Machines/move_to_box \
    --single-task "Pick up the cube and place it into the box." \
    --robot koch_follower



In [ ]:
!pip install gymnasium imageio torch gym_pusht


In [ ]:
!pip install gym

In [ ]:
!python "/content/movetobox.py"


In [ ]:
!python -m lerobot.scripts.visualize_dataset \
    --repo-id=lerobot/pusht \
    --episode-index=0 \
    --output-dir=/content/lerobot/outputs/visualized_episode \
    --save=1 \
    --mode=rgb



In [ ]:
!python -m lerobot.scripts.visualize_dataset \
    --repo-id lerobot/pusht \
    --root ./my_local_data_dir \
    --local-files-only 1 \
    --episode-index 0

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli upload ${HF_USER}/my_smolvla \
  /content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model

In [ ]:
from huggingface_hub import hf_hub_download

# Download the SEPO fine-tuned model checkpoint from Hugging Face Hub
ckpt_path = hf_hub_download(
    repo_id="Xssama/SEPO_DNA",
    filename="finetuned_sepo_kl.ckpt",  # or "finetuned_sepo_kl_gf.ckpt"
    cache_dir="./checkpoints"  # optional: local cache folder
)

print(f"Checkpoint downloaded to: {ckpt_path}")


In [ ]:
!git clone https://github.com/ozekri/SEPO.git


In [ ]:
!find SEPO -name "*.py"


In [ ]:
sys.path.append('./SEPO/src')
from model import SEPOModel


In [ ]:
import sys
sys.path.append('./SEPO')  # Add SEPO repo folder to path

from src.model import SEPOModel  # or correct import path from the repo

# Now load checkpoint and use model as explained earlier
checkpoint_path = "/content/checkpoints/models--Xssama--SEPO_DNA/snapshots/a531d9b9c8ee0134b426d7a3543fdd4d1d50bccf/finetuned_sepo_kl.ckpt"
model = SEPOModel()
checkpoint = torch.load(checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['state_dict'])
model.eval()

# Dummy input
input_tensor = torch.randn(1, 3, 224, 224)
with torch.no_grad():
    output = model(input_tensor)

print(output)



In [ ]:
!git clone https://huggingface.co/Frontier-Machines/move_to_box

In [ ]:
cd move_to_box


In [ ]:
ls


In [ ]:
ls train/


In [ ]:
cat config.json


In [ ]:
!python train/train.py --config config.json


In [ ]:
ls -R /content/move_to_box/train


In [ ]:
ls -R /content/move_to_box


In [ ]:
!git clone https://huggingface.co/Frontier-Machines/move_to_duct_tape

In [ ]:
cd move_to_duct_tape

In [ ]:
ls

In [ ]:
!head -40 /content/move_to_box/move_to_duct_tape/README.md


In [ ]:
!ls /content/lerobot

In [ ]:
!python /content/lerobot/src/lerobot/scripts/train.py --config /content/move_to_box/move_to_duct_tape/config.json

